In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import random


# Creating an instance
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


from selenium.webdriver.common.by import By
# Logging into LinkedIn
     # this will open the link


In [6]:
CEC_names = pd.read_csv('profiles_final_all.csv')
CEC_names = CEC_names['Name']
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [3]:
# Logging into LinkedIn
def login():
    driver.get(f"https://linkedin.com/uas/login")
    time.sleep(3)
  
    username = driver.find_element(By.ID,"Email")
    username.send_keys("GMUGI8@gmail.com")  # Enter Your Email Address
  
    pword = driver.find_element(By.ID, "Password")
    pword.send_keys("#GMUGI888")        # Enter Your Password
  
    driver.find_element(By.XPATH,"//button[@type='Sign in']").click()

In [7]:
link_list = []
names = []
missing_count = 0
check = 10

#currentCompany=%5B%225489%22%5D&
#Collect linkedin profile links
for i in range(0,len(CEC_names)):
    search_name = CEC_names[i].replace(" ","%20")
    print(search_name)
    driver.get("https://www.linkedin.com/search/results/people/?keywords="+search_name+"%20George%20Mason&origin=FACETED_SEARCH&sid=RLp")
    time.sleep(random.randint(1, 5))
    src = driver.page_source
  
       # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')
    #print(soup.prettify())
    names.append(CEC_names[i])
    #Navigate HTML code
    result = soup.find("div", {"class": "t-roman t-sans"})
    if result:
        link = result.find("a").get('href')
        name = result.find("span", {"dir": "ltr"})
        if name:
            name = name.find("span", {"aria-hidden": "true"}).get_text()
            name_split = search_name.split("%20")
            print(name)
            #verify the taken link is a profile then add to list
            if '/in/' in link and any(word in name for word in name_split):
                link_list.append(link)
                print('True')
            else:
                link_list.append(None)
                missing_count += 1
        else:
            link_list.append(None)
            missing_count += 1
    else:
        link_list.append(None)
        missing_count += 1
    
    print(link)

Sai%20Manoj%20Pudukotai%20Dinakarrao
https://www.linkedin.com/in/robert-osgood-b5a32114?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAALlEzYBvrMBqMSEXuCoUAdQr3lEU0fnwQo
Tolga%20Soyata
Tolga Soyata
True
https://www.linkedin.com/in/tolga-soyata-3405244?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAADJ-uABTCofc7hEjf-Yjgnek5WmkmUqnuw
Weiwen%20Jiang
Weiwen Jiang
True
https://www.linkedin.com/in/weiwen-jiang-15356a165?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACdqdk8BbZn5lhX4qgj4IOyA3__pMIUx3L0
Xiang%20Chen
Xiang Chen
True
https://www.linkedin.com/in/xiang-chen-36219b36?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAeAkToBQc6TLcutV0Xewad_HPzDRTV9Xp4
Xuan%20Wang
Xuan Wang
True
https://www.linkedin.com/in/xuan-wang-38819610a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABtvb5gBK0-b0r3bZ64FwnUzE0T3kjtcMgk
Zhi%20Tian
Zhi (Gerry) Tian
True
https://www.linkedin.com/in/zhi-gerry-tian-30abb622?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAATOCToBpvznetCImlaVBT0vV2Dfln7LpS4
Ali%20Raz
A

In [8]:
temp = {'Name': names, 'Link': link_list} 
       
df = pd.DataFrame(temp)
df.to_csv('LinkdeIn_Links_2.csv')

In [24]:
#function that scrolls to bottom of webpage, to load entire webpage.
def scroll():
    start = time.time()
  
    # will be used in the while loop
    initialScroll = 0
    finalScroll = 500
  
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll 
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 500
  
        # we will stop the script for 3 seconds so that 
        # the data can load
        time.sleep(3)
        # You can change it as per your needs and internet speed
  
        end = time.time()
  
        # We will scroll for 20 seconds.
        # You can change it as per your needs and internet speed
        if round(end - start) > 2:
            break

In [25]:
#Obtain the information for each publication published on each profile
def get_publications(fname):
    #collect HTML code
    soup = BeautifulSoup(src, 'lxml')
    #Create data lists
    titles = []
    descriptions = []
    dates = []
    names = []
    #Navigate HTML code
    pub_list = soup.find("ul", {"class": "pvs-list"})
    publications = pub_list.find_all("div", {"class": "display-flex flex-column full-width align-self-center"})
    count = len(publications)
    #Add each piece of data to the appropriate data lists.
    if publications:
        for post in publications:
            names.append(fname)
            temp = post.find("div", {"class": "display-flex flex-row justify-space-between"})
            #Get publication title
            if temp:
                title = temp.find("span", {"class": "mr1 t-bold"})
                if title:
                    title = title.find("span", {"class": "visually-hidden"})
                    if title:
                        title = title.get_text()
                        titles.append(title)
                #Get publication Date
                date = temp.find("span", {"class": "t-14 t-normal"})
                if date:
                    date = date.find("span", {"class": "visually-hidden"})
                    if date:
                        date = date.get_text()
                        date = date.split(" · ")[-1]
                        #print(date)
                        dates.append(date)
                else:
                    dates.append(None)
            else:
                titles.append(None)
                dates.append(None)

            temp = post.find("div", {"class": "display-flex full-width"})
            #Get publication description
            if temp:
                description = temp.find("span", {"class": "visually-hidden"})
                if description:
                    description = description.get_text()
                descriptions.append(description)
            else:
                descriptions.append(None)
    temp = {'Name': names, 'Title': titles, 'Descriptions': descriptions, 'Date': dates}
    #print(temp)
    #Combine lists into a dataframe   
    df = pd.DataFrame(temp)
    print(df)
    return df
    df.to_csv('LinkdeIn_Publications.csv')

In [29]:
#Get the about section form each profile if there is an about section
profiles = pd.read_csv('LinkdeIn_Links_2.csv')
profiles = profiles.dropna(how='any',axis=0).reset_index()
Names = profiles['Name']
links = profiles['Link']
url_pub = "details/publications/"
abouts = []
temp_nam = []
results = []
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

In [ ]:
login()

In [30]:
for i in range(0,len(links)):
    driver.get(links[i])
    scroll()
    src = driver.page_source
  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')
    main = soup.find("main", {"id": "main"})
    #Get about section from the selected profile
    if main:
        text = main.get_text()
        if "AboutAbout" in text:
            about = text.split('AboutAbout')[1]
            about = about.split('ExperienceExperience')[0]
            about = about.split('EducationEducation')[0]
            about = about.split('ActivityActivity')[0]
            about = about.replace('\n','')
            about = about.replace('…see more','')
            print(about)
        else:
            about = None
    else:
        about = None
    abouts.append(about)
    temp_nam.append(Names[i])
    driver.get(driver.current_url+url_pub)
    time.sleep(3)
    scroll()
    src = driver.page_source
    df = get_publications(Names[i])
    results.append(df)
final_result = pd.concat(results)
temp = {'Name': temp_nam, 'About': abouts}
df1 = pd.DataFrame(temp)
df1.to_csv('LinkdeIn_Abouts.csv')
final_result.to_csv('LinkdeIn_Publications.csv')
    

Empty DataFrame
Columns: [Name, Title, Descriptions, Date]
Index: []
   I am a passionate and driven Human Resource (HR) professional with over five years in the field. My experience includes talent acquisition, training and development, human resource information systems (HRIS), applicant tracking systems, and data analysis. I am highly skilled in coordinating daily operational and office procedures while maintaining a high level of customer focus and confidentiality by building strong working relationships at all levels. I possess an outstanding ability to multi-task and prioritize tasks while keeping an open mind and providing excellent customer service when executing responsibilities.I am a passionate and driven Human Resource (HR) professional with over five years in the field. My experience includes talent acquisition, training and development, human resource information systems (HRIS), applicant tracking systems, and data analysis. I am highly skilled in coordinating daily opera

In [31]:
#Save results
final_result = pd.concat(results)
temp = {'Name': temp_nam, 'About': abouts}
df1 = pd.DataFrame(temp)
df1.to_csv('LinkdeIn_Abouts.csv')
final_result.to_csv('LinkdeIn_Publications.csv')